データ読み込み

In [1]:
from sklearn.datasets import load_breast_cancer#サンプルデータセットのライブラリ
import pandas as pd#テーブルデータや時系列データを操作するためのデータ構造と演算のライブラリ

data = load_breast_cancer()#データ読み込み
data#dataを表示

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

目的変数と説明変数のテーブルデータに変換


In [2]:
train_x = pd.DataFrame(data=data.data,columns=data.feature_names)#説明変数のテーブルデータに変換。データ:data,カラム:columns
train_x.head()#train_xのデータフレームの最初の5行表示

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
train_y = pd.DataFrame(data=data.target,columns=['class'])#目的変数のテーブルデータに変換。データ:data,カラム:columns
train_y.head()#train_yのデータフレームの最初の5行表示

,class
0,0
1,0
2,0
3,0
4,0


In [4]:
from sklearn.metrics import log_loss, accuracy_score#混同行列を計算するライブラリ
from sklearn.model_selection import KFold#ホールドアウト、交差検証(cross validation)するライブラリ
import lightgbm as lgb#LightGBMのライブラリ
import numpy as np#数値計算するライブラリ

# 各foldのスコアを保存する空のリスト
scores_accuracy = []#[]:正解率用のリスト
scores_logloss = []#[]:評価関数LogLoss用のリスト

# クロスバリデーション(k-fold cross validation)を行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=3, shuffle=True, random_state=42)#n_splits:分割,shuffle:シャッフルするかどうか,random_state:何度実行しても同じランダムな分割になるように固定する値。指定しないと毎回分割データが変わり結果が異なってしまう

確認用【for tr_idx, va_idx in kf.split(train_x):】

In [5]:
cnt = 1#カウント用の数字
for tr_idx, va_idx in kf.split(train_x):#クロスバリデーションで分割。for：繰り返し(n_splitsの回数繰り返す),in：どんな条件で繰り返すか
    print('########{}回目:データ数 tr:{},va:{}########'.format(cnt,len(tr_idx),len(va_idx)))#print:文字列を出力、len([]):リストの長さ
    print('tr_idx:{}'.format(tr_idx))#'{}'.format(変数)
    print('va_idx:{}'.format(va_idx))
    cnt += 1#cntを1足す。

########1回目:データ数 tr:379,va:190########
tr_idx:[  1   3   4   5   8  12  13  14  16  17  20  21  23  24  25  26  27  28
  31  32  34  35  36  37  38  40  41  42  43  44  45  47  48  49  50  51
  52  53  56  57  58  59  61  62  64  65  66  67  71  74  80  85  87  91
  92  94  95  96  97  98  99 100 102 103 105 106 107 111 112 113 115 116
 119 120 121 122 123 125 126 127 128 129 130 133 134 135 136 138 139 142
 143 146 147 150 151 152 154 156 157 159 160 161 162 164 166 168 169 170
 171 173 174 175 178 179 180 183 186 187 189 190 191 193 194 197 198 200
 201 202 205 206 207 212 213 214 215 216 217 218 219 220 221 223 224 225
 226 229 230 232 233 234 236 237 239 240 241 242 243 246 251 252 253 254
 256 258 259 260 262 263 266 267 268 269 270 272 273 276 277 278 279 280
 282 283 285 286 288 289 291 292 293 294 295 296 297 299 300 301 302 303
 304 306 307 308 309 311 312 313 314 315 316 317 318 319 321 323 324 325
 326 327 328 330 335 336 337 338 339 340 342 343 344 345 346 347 348 349
 350 

初期ハイパーパラメータ初期設定

In [6]:
for tr_idx, va_idx in kf.split(train_x):#クロスバリデーションで分割。for：繰り返し(n_splitsの回数繰り返す),in：どんな条件で繰り返すか
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]#iloc[]:行番号、列番号を指定して要素を取り出す
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = lgb.LGBMClassifier() # LightGBMのモデルを定義。分類:LGBMClassifier(),回帰:LGBMRegressor()
    model.fit(tr_x, tr_y) # モデルの学習:fit(説明変数,目的変数)

    # テストデータの予測クラス (予測クラス(0 or 1)を返す)
    y_pred = model.predict(va_x)
    # テストデータのクラス予測確率 (各クラスの予測確率 [クラス0の予測確率,クラス1の予測確率] を返す)
    y_pred_prob = model.predict_proba(va_x)    

    # モデル評価
    # acc : 正答率
    accuracy = accuracy_score(va_y,y_pred)#accuracy_score(正解値,予測値)
    print('accuracy :',accuracy)

    # 評価関数LogLoss 
    logloss =  log_loss(va_y,y_pred_prob) # 引数 : log_loss(正解クラス,[クラス0の予測確率,クラス1の予測確率])
    print('logloss :', logloss)

    # 各foldのスコアを保存する
    scores_logloss.append(logloss)#append:listに追加
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
print('scores_logloss:{}'.format(scores_logloss))
print('scores_accuracy:{}'.format(scores_accuracy))
logloss = np.mean(scores_logloss)#各foldの評価関数LogLossの平均値
accuracy = np.mean(scores_accuracy)#各foldの正解率の平均値
print("===予測モデルの評価指標(平均スコア)===")
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')#f'':変数をそのまま指定できる,{*.4f}:小数点以下の桁数(この場合4桁)

C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.9526315789473684
logloss : 0.16737458759897722


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.9631578947368421
logloss : 0.16552806778699025


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy : 0.9629629629629629
logloss : 0.17521848393273715
scores_logloss:[0.16737458759897722, 0.16552806778699025, 0.17521848393273715]
scores_accuracy:[0.9526315789473684, 0.9631578947368421, 0.9629629629629629]
===予測モデルの評価指標(平均スコア)===
logloss: 0.1694, accuracy: 0.9596


グリッドリサーチでハイパーパラメータ設定

In [7]:
from sklearn.model_selection import GridSearchCV#グリッドサーチのライブラリ

model = lgb.LGBMClassifier() # LightGBMのモデルを定義
kf = KFold(n_splits=3, shuffle=True, random_state=42)#n_splits:分割,shuffle:シャッフルするかどうか,random_state:何度実行しても同じランダムな分割になるように固定する値。指定しないと毎

# パラメーターを設定する
param_grid = {"max_depth": [5,6,7,8,9],#木構造の深さを限定するための変数．データが少ないときに過学習を防ぐために設定する
              "learning_rate" : [0.05,0.06,0.07,0.08,0.09],#学習率
              'num_leaves': [5,6,7,8,9]#木にある分岐の個数
             }
# パラメータチューニングをグリッドサーチで行うために設定する
## このGridSearchCV には注意が必要 scoring は そのスコアを基準にして最適化する
grid = GridSearchCV(estimator = model,#モデル
                    param_grid = param_grid,#パラメータ
                    scoring = 'accuracy',#評価指標
                    cv = kf,#交差検定の回数
                    verbose=1,#verbose=1:一定の間隔でログ表示，verbose=2:テスト毎にログ表示，verbose=3:テスト毎にスコアも含めてログ表示
                    return_train_score = True,#False場合、トレーニングスコアを含まない
                    n_jobs = -1)#同時実行数(-1にするとコア数で同時実行)。#コア数とはコンピュータのCPUに内蔵された稼働するプロセッサコアの数。 コアの数だけ複数のプログラムを並列に動作させられる

grid.fit(train_x, train_y)#学習

print("===予測モデルの評価指標(平均スコア)===")
print("最も良いパラメータ:{}".format(grid.best_params_))
print(f'accuracy: {grid.best_score_:.4f}')#f'':変数をそのまま指定できる,{*.4f}:小数点以下の桁数(この場合4桁)

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:   51.0s finished
C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


===予測モデルの評価指標(平均スコア)===
最も良いパラメータ:{'learning_rate': 0.09, 'max_depth': 5, 'num_leaves': 8}
accuracy: 0.9649


アンサンブル学習(https://potesara-tips.com/ensemble-voting/#toc10)

In [8]:
import xgboost as xgb#機械学習モデル：xgboost
import lightgbm as lgb#機械学習モデル：lightgbm
from catboost import CatBoost#機械学習モデル：catboost
from catboost import Pool

In [9]:
def xgb_train(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv, X_test, y_test, loop_counts):
    # データを格納する
    # 学習用
    xgb_train = xgb.DMatrix(X_train_cv, label=y_train_cv)
    # 検証用
    xgb_eval = xgb.DMatrix(X_eval_cv, label=y_eval_cv)
    # テスト用
    xgb_test = xgb.DMatrix(X_test, label=y_test)
    
    # パラメータを設定
    xgb_params = {
        'objective': 'multi:softprob',  # 多値分類問題
        'num_class': 3,                 # 目的変数のクラス数
        'learning_rate': 0.1,           # 学習率
        'eval_metric': 'mlogloss'       # 学習用の指標 (Multiclass logloss)
    }

    # 学習
    evals = [(xgb_train, 'train'), (xgb_eval, 'eval')] # 学習に用いる検証用データ
    evaluation_results = {}                            # 学習の経過を保存する箱
    bst = xgb.train(xgb_params,                        # 上記で設定したパラメータ
                    xgb_train,                         # 使用するデータセット
                    num_boost_round=200,               # 学習の回数
                    early_stopping_rounds=10,          # アーリーストッピング
                    evals=evals,                       # 学習経過で表示する名称
                    evals_result=evaluation_results,   # 上記で設定した検証用データ
                    verbose_eval=0                     # 学習の経過の表示(非表示)
                    )
    
    # テストデータで予測
    y_pred = bst.predict(xgb_test, ntree_limit=bst.best_ntree_limit)
    y_pred_max = np.argmax(y_pred, axis=1)

    print('Trial: ' + str(loop_counts))
    accuracy = accuracy_score(y_test.values.tolist(), y_pred_max)
    print('XGBoost Accuracy:', accuracy)
    
    return(bst, y_pred_max, accuracy)#return:出力、bst:モデル、y_pred_max:予測値,acuracy:正解率

In [10]:
def lgbm_train(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv, X_test, y_test):
    # データを格納する
    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           free_raw_data=False)
    
    # パラメータを設定
    params = {'task': 'train',                # レーニング ⇔　予測predict
              'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'multiclass',      # 目的関数：多値分類、マルチクラス分類
              'metric': 'multi_logloss',      # 検証用データセットで、分類モデルの性能を測る指標
              'num_class': 3,                 # 目的変数のクラス数
              'learning_rate': 0.1,           # 学習率（初期値0.1）
              'num_leaves': 23,               # 決定木の複雑度を調整（初期値31）
              'min_data_in_leaf': 1,          # データの最小数（初期値20）
              'verbose': -1,
             }

    # 学習
    evaluation_results = {}                                # 学習の経過を保存する箱
    model = lgb.train(params,                              # 上記で設定したパラメータ
                      lgb_train,                           # 使用するデータセット
                      num_boost_round=200,                 # 学習の回数
                      valid_names=['train', 'valid'],      # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],    # モデルの検証に使用するデータセット
                      evals_result=evaluation_results,     # 学習の経過を保存
                      early_stopping_rounds=10,            # アーリーストッピングの回数
                      verbose_eval=0)                      # 学習の経過を表示する刻み（非表示）

    # テストデータで予測
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_pred_max = np.argmax(y_pred, axis=1)
    # Accuracy の計算
    accuracy = accuracy_score(y_test, y_pred_max)
    #accuracy = sum(y_test == y_pred_max) / len(y_test)
    print('LightGBM Accuracy:', accuracy)
    
    return(model, y_pred_max, accuracy)#return:出力、model:モデル、y_pred_max:予測値,acuracy:正解率

In [11]:
def catboost_train(X_train_cv, y_train_cv, X_eval_cv, y_eval_cv, X_test, y_test):
    # データを格納する
    # 学習用
    CatBoost_train = Pool(X_train_cv, label=y_train_cv)
    # 検証用
    CatBoost_eval = Pool(X_eval_cv, label=y_eval_cv)
    
    # パラメータを設定
    params = {        
        'loss_function': 'MultiClass',    # 多値分類問題
        'num_boost_round': 1000,          # 学習の回数
        'early_stopping_rounds': 10       # アーリーストッピングの回数
    }
    
    # 学習
    catb = CatBoost(params)
    catb.fit(CatBoost_train, eval_set=[CatBoost_eval], verbose=False)

    # テストデータで予測
    y_pred = catb.predict(X_test, prediction_type='Probability')
    y_pred_max = np.argmax(y_pred, axis=1)

    # Accuracy の計算
    accuracy = accuracy_score(y_test, y_pred_max)
    #accuracy = sum(y_test == y_pred_max) / len(y_test)
    print('CatBoost Accuracy:', accuracy)
    
    return(catb, y_pred_max, accuracy)#return:出力、catb:モデル、y_pred_max:予測値,acuracy:正解率

In [107]:
import xgboost as xgb#機械学習モデル：xgboost
import lightgbm as lgb#機械学習モデル：lightgbm
from catboost import CatBoost#機械学習モデル：catboost
from catboost import Pool
from sklearn.model_selection import train_test_split

# Voting
# 各5つのモデルを保存するリストの初期化
xgb_models = []
lgbm_models = []
catb_models = []
# 各5つのモデルの正答率を保存するリストの初期化
xgb_accuracies = []
lgbm_accuracies = []
catb_accuracies = []
# 学習のカウンター
loop_counts = 1


# 学習データとテストデータに分ける
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=train_y)

# 各5つのモデルの予測を保存する配列の初期化（5seed*5cv*3モデル）
first_preds = np.zeros((len(y_test), 5*3*3))


# ５つのシード値で予測
for seed_no in range(5): 
        
    # 学習データの数だけの数列（0行から最終行まで連番）
    row_no_list = list(range(len(y_train)))

    # KFoldクラスをインスタンス化（これを使って3分割する）
    K_fold = StratifiedKFold(n_splits=3, shuffle=True,  random_state= seed_no)

    # KFoldクラスで分割した回数だけ実行（ここでは5回）
    for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
        # ilocで取り出す行を指定
        X_train_cv = X_train.iloc[train_cv_no, :]
        #y_train_cv = pd.Series(y_train).iloc[train_cv_no]
        y_train_cv = y_train.iloc[train_cv_no]
        X_eval_cv = X_train.iloc[eval_cv_no, :]
        #y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
        y_eval_cv = y_train.iloc[eval_cv_no]
        
        # XGBoostの学習を実行
        bst, bst_pred, bst_accuracy = xgb_train(X_train_cv, y_train_cv,
                                                X_eval_cv, y_eval_cv, 
                                                X_test, y_test,
                                                loop_counts)
        # LIghtGBMの学習を実行
        model, model_pred, model_accuracy = lgbm_train(X_train_cv, y_train_cv,
                                                       X_eval_cv, y_eval_cv,
                                                       X_test, y_test)
        # CatBoostの学習を実行
        catb, catb_pred, catb_accuracy = catboost_train(X_train_cv, y_train_cv,
                                                        X_eval_cv, y_eval_cv,
                                                        X_test, y_test)
        
        # 学習が終わったモデルをリストに入れておく
        xgb_models.append(bst) 
        lgbm_models.append(model) 
        catb_models.append(catb) 
               
        # 学習が終わったモデルの正答率をリストに入れておく
        xgb_accuracies.append(bst_accuracy) 
        lgbm_accuracies.append(model_accuracy) 
        catb_accuracies.append(catb_accuracy) 
               
        # 学習が終わったモデルの予測をリストに入れておく
        first_preds[:, loop_counts-1] = bst_pred
        first_preds[:, loop_counts-1 + 15] = model_pred
        first_preds[:, loop_counts-1 + 30] = catb_pred
        
        # 実行回数のカウント
        loop_counts += 1        



C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 1
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 1.0
Trial: 2
XGBoost Accuracy: 1.0
LightGBM Accuracy: 0.9722222222222222
CatBoost Accuracy: 0.9444444444444444
Trial: 3
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 0.9722222222222222


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 4
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9166666666666666
CatBoost Accuracy: 0.9722222222222222
Trial: 5
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9722222222222222
CatBoost Accuracy: 0.9444444444444444
Trial: 6
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9166666666666666
CatBoost Accuracy: 1.0


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 7
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 0.9722222222222222
Trial: 8
XGBoost Accuracy: 0.9166666666666666
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 0.9444444444444444
Trial: 9
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9722222222222222
CatBoost Accuracy: 0.9722222222222222


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 10
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.8888888888888888
CatBoost Accuracy: 1.0
Trial: 11
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9722222222222222
CatBoost Accuracy: 0.9722222222222222
Trial: 12
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 0.9444444444444444


C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 13
XGBoost Accuracy: 0.9722222222222222
LightGBM Accuracy: 0.9722222222222222
CatBoost Accuracy: 1.0
Trial: 14
XGBoost Accuracy: 0.9444444444444444
LightGBM Accuracy: 0.8888888888888888
CatBoost Accuracy: 0.9722222222222222
Trial: 15
XGBoost Accuracy: 1.0
LightGBM Accuracy: 0.9444444444444444
CatBoost Accuracy: 0.9722222222222222


In [15]:
import xgboost as xgb#機械学習モデル：xgboost
import lightgbm as lgb#機械学習モデル：lightgbm
from catboost import CatBoost#機械学習モデル：catboost
from catboost import Pool
from sklearn.model_selection import StratifiedKFold

# Voting
# 各5つのモデルを保存するリストの初期化
xgb_models = []
lgbm_models = []
catb_models = []
# 各5つのモデルの正答率を保存するリストの初期化
xgb_accuracies = []
lgbm_accuracies = []
catb_accuracies = []
# 学習のカウンター
loop_counts = 1


# 各5つのモデルの予測を保存する配列の初期化（3cv*3モデル）
first_preds = np.zeros((len(y_test), 3*3))

# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って3分割する）
K_fold = StratifiedKFold(n_splits=3, shuffle=True,  random_state= 1)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    #y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    y_train_cv = y_train.iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    #y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    y_eval_cv = y_train.iloc[eval_cv_no]

    # XGBoostの学習を実行
    bst, bst_pred, bst_accuracy = xgb_train(X_train_cv, y_train_cv,
                                            X_eval_cv, y_eval_cv, 
                                            X_test, y_test,
                                            loop_counts)
    # LIghtGBMの学習を実行
    model, model_pred, model_accuracy = lgbm_train(X_train_cv, y_train_cv,
                                                   X_eval_cv, y_eval_cv,
                                                   X_test, y_test)
    # CatBoostの学習を実行
    catb, catb_pred, catb_accuracy = catboost_train(X_train_cv, y_train_cv,
                                                    X_eval_cv, y_eval_cv,
                                                    X_test, y_test)

    # 学習が終わったモデルをリストに入れておく
    xgb_models.append(bst) 
    lgbm_models.append(model) 
    catb_models.append(catb) 

    # 学習が終わったモデルの正答率をリストに入れておく
    xgb_accuracies.append(bst_accuracy) 
    lgbm_accuracies.append(model_accuracy) 
    catb_accuracies.append(catb_accuracy) 

    # 学習が終わったモデルの予測をリストに入れておく
    first_preds[:, loop_counts-1] = bst_pred
    first_preds[:, loop_counts-1 + 3] = model_pred
    first_preds[:, loop_counts-1 + 6] = catb_pred

    # 実行回数のカウント
    loop_counts += 1        



C:\Users\s180168\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:104: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


Trial: 1
XGBoost Accuracy: 0.9736842105263158
LightGBM Accuracy: 0.9649122807017544
CatBoost Accuracy: 0.9736842105263158
Trial: 2
XGBoost Accuracy: 0.9824561403508771
LightGBM Accuracy: 0.9385964912280702
CatBoost Accuracy: 0.956140350877193
Trial: 3
XGBoost Accuracy: 0.956140350877193
LightGBM Accuracy: 0.956140350877193
CatBoost Accuracy: 0.9649122807017544


In [16]:
# 単独のモデルでの、テストデータの正答率
print('XGBoost Accuracy: ', np.array(xgb_accuracies).mean())#平均値算出
print('LightGBM Accuracy: ', np.array(lgbm_accuracies).mean())#平均値算出
print('CatBoost Accuracy: ', np.array(catb_accuracies).mean())#平均値算出

XGBoost Accuracy:  0.9707602339181287
LightGBM Accuracy:  0.9532163742690059
CatBoost Accuracy:  0.9649122807017544


In [24]:
# アンサンブルのモデルでの、テストデータの正答率
import itertools
# データ格納用のnumpy行列を作成
first_preds_max = pd.DataFrame(np.zeros((len(y_test), 3)))

# 予測したクラスのデータをpandas.DataFrameに入れる
df_first_preds = pd.DataFrame(first_preds)

# 各列（0,1,2）に、そのクラスを予測したモデルの数を入れる
first_preds_max.iloc[:, 0] = (df_first_preds == 0).sum(axis=1)
first_preds_max.iloc[:, 1] = (df_first_preds == 1).sum(axis=1)
first_preds_max.iloc[:, 2] = (df_first_preds == 2).sum(axis=1)

# 各行で、そのクラスを予測したモデルの数が最も多いクラスを得る
pred_max = np.argmax(np.array(first_preds_max), axis=1)

# Accuracy を計算する
#accuracy = sum(y_test.values.tolist() == pred_max) / len(y_test)
accuracy = accuracy_score(y_test, pred_max)
print('accuracy:', accuracy)
#df_accuracy = pd.DataFrame({'va_y': list(itertools.chain.from_iterable(y_test.values)),
#                            'y_pred_max': pred_max})
#print(pd.crosstab(df_accuracy['va_y'], df_accuracy['y_pred_max']))

accuracy: 0.9736842105263158
